In [7]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


train_file_path = "train.csv"
df_train = pd.read_csv(train_file_path)

q1 = df_train['SalePrice'].quantile(0.25)
q3 = df_train['SalePrice'].quantile(0.75)

df_train['PriceCategory'] = pd.cut(df_train['SalePrice'], bins=[-np.inf, q1, q3, np.inf], labels=['Económicas', 'Intermedias', 'Caras'])



# y como variable categórica
y = df_train['PriceCategory']
X = df_train.select_dtypes(include=np.number).drop(['SalePrice'], axis=1)
X = X.replace([np.inf, -np.inf], np.nan).dropna()
y = y.loc[X.index]

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)

# Modelo 1: kernel lineal
svm_linear = SVC(kernel='linear', C=1)
svm_linear.fit(X_train, y_train)
print("🔸 Kernel lineal\n", classification_report(y_test, svm_linear.predict(X_test)))

# Modelo 2: kernel RBF (radial) con gamma ajustado
svm_rbf = SVC(kernel='rbf', C=10, gamma=0.01)
svm_rbf.fit(X_train, y_train)
print("🔸 Kernel RBF\n", classification_report(y_test, svm_rbf.predict(X_test)))

# Modelo 3: kernel polinomial grado 3
svm_poly = SVC(kernel='poly', C=5, gamma='scale', degree=3)
svm_poly.fit(X_train, y_train)
print("🔸 Kernel polinomial (grado 3)\n", classification_report(y_test, svm_poly.predict(X_test)))


🔸 Kernel lineal
               precision    recall  f1-score   support

       Caras       0.90      0.86      0.88        88
  Económicas       0.85      0.89      0.87        81
 Intermedias       0.88      0.88      0.88       168

    accuracy                           0.88       337
   macro avg       0.88      0.88      0.88       337
weighted avg       0.88      0.88      0.88       337

🔸 Kernel RBF
               precision    recall  f1-score   support

       Caras       0.92      0.88      0.90        88
  Económicas       0.83      0.85      0.84        81
 Intermedias       0.86      0.88      0.87       168

    accuracy                           0.87       337
   macro avg       0.87      0.87      0.87       337
weighted avg       0.87      0.87      0.87       337

🔸 Kernel polinomial (grado 3)
               precision    recall  f1-score   support

       Caras       0.92      0.76      0.83        88
  Económicas       0.85      0.77      0.81        81
 Intermedias 

# INFORME

## Resultados del modelo SVM
Se entrenaron tres modelos de Máquina de Vectores de Soporte (SVM) utilizando diferentes kernels: lineal, radial (RBF) y polinomial de grado 3, con el objetivo de predecir la categoría del precio de las viviendas (Económicas, Intermedias o Caras).

🔸 1. SVM con Kernel Lineal
Precisión promedio: 88%

Rendimiento equilibrado entre las tres categorías, con f1-scores de 0.88 para “Intermedias”, 0.87 para “Económicas” y 0.88 para “Caras”.

Este modelo muestra un comportamiento robusto y balanceado, capturando bien los patrones lineales entre las variables.

🔸 2. SVM con Kernel RBF (Radial Basis Function)
Precisión promedio: 87%

Ligeramente menor en precisión global, pero aún con un rendimiento competitivo.

Obtuvo su mejor desempeño para la clase “Caras” con un f1-score de 0.90.

Este kernel puede capturar relaciones no lineales, aunque en este caso no superó al kernel lineal, lo cual sugiere que los datos podrían ser aproximadamente separables linealmente.

🔸 3. SVM con Kernel Polinomial (grado 3)
Precisión promedio: 83%

Fue el modelo con menor desempeño global.

Mostró una pérdida notable en recall para las clases “Caras” y “Económicas”, lo que indica que no predijo correctamente varios ejemplos de estas categorías.

Aunque aumentó el recall en la clase “Intermedias” (0.90), lo hizo a costa de reducir el rendimiento en las otras clases.

## Conclusiones

* El modelo SVM con kernel lineal fue el que mejor desempeño mostró, con una precisión general del 88% y un balance sólido entre precisión y recall en las tres clases. Esto sugiere que la separación entre categorías de precios puede resolverse adecuadamente mediante fronteras lineales, sin necesidad de introducir transformaciones no lineales más complejas como las que ofrecen los kernels RBF o polinomial.

